In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [69]:
cfg = Config(
#     n_mels = 128,
#     f_min = 20,
#     f_max = 16000,
#     n_fft = 2048,
#     hop_length = 512,
    
    noise_nsr_dbs = [35, 30, 20, 10],
    site='SSW', 
    use_neptune=True, 
    n_epochs=15, 
    bs=32, 
    lr=1e-4, 
    model='resnest50-mixed-spec',
    scheduler='torch.optim.lr_scheduler.OneCycleLR'
)

In [70]:
cfg.scheduler_params

{'max_lr': 0.001, 'steps_per_epoch': 336, 'epochs': 15}

In [71]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [35, 30, 20, 10],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.0001,
 'n_epochs': 15,
 'model': 'resnest50-mixed-spec',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.OneCycleLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
 

In [72]:
prep = MixedSig2Spec(cfg)
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# 

main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

# posp = torch.nn.Sequential(
#     torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
#     torch.nn.ReLU(),
#     torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)
# )

posp = torch.nn.Linear(in_features=2048, 
                       out_features=len(cfg.labels), bias=True)

# for param in posp.parameters():
#     param.requires_grad = True

main_model.fc = posp

model = torch.nn.Sequential(prep, main_model)

# for param in model.parameters():
#     param.requires_grad = False
# for m in [model[1].layer3, model[1].layer4, model[1].avgpool, model[1].fc]:
#     for param in m.parameters():
#         param.requires_grad = True


model = model.cuda()




# model

# model = torch.nn.Sequential(Sig2Spec(cfg, forward_as_image=True), preprocess, rn_model)
# model[1].fc = torch.nn.Linear(in_features=2048, out_features=len(cfg.labels), bias=True)

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [73]:
# model[1].layer3, model[1].layer4, model[1].avgpool, model[1].fc

In [74]:
lrn = Learner("resnest50-mixed-spec", cfg, model)

In [75]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-61


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-03 15:55:04 epoch:  14 train loss: 0.111602 train f1: 0.567985 valid loss: 0.083654 valid f1: 0.369352


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-03 15:57:25 epoch:  15 train loss: 0.111643 train f1: 0.573540 valid loss: 0.086335 valid f1: 0.333333


In [76]:
lrn.scheduler.state_dict()

{'total_steps': 5040,
 '_schedule_phases': [{'end_step': 1511.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 5039,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7fbe204b0d00>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [4e-05],
 'last_epoch': 5040,
 '_step_count': 5041,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.198235228265301e-09]}

In [77]:
# lrn.name = 'resnest50-mixed-spec-half-frozen-20210502-144336-latest'

In [78]:
# lrn.save_checkpoint(50, 0.097231, 0.234157)

In [79]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,438,30541,2289,332,0.250500
2,0.2,309,32405,425,461,0.410904
3,0.3,244,32644,186,526,0.406667
4,0.4,194,32731,99,576,0.365005
5,0.5,167,32765,65,603,0.333333
6,0.6,140,32789,41,630,0.294427
7,0.7,117,32802,28,653,0.255738
8,0.8,108,32814,16,662,0.241611
9,0.9,83,32823,7,687,0.193023


In [80]:
lrn.load_checkpoint()

{'epoch': 13,
 'valid_loss': 0.08365363029665068,
 'valid_score': 0.36935165524482727}

In [81]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,470,30559,2271,300,0.267730
2,0.2,348,32412,418,422,0.453125
3,0.3,275,32654,176,495,0.450450
4,0.4,227,32722,108,543,0.410860
5,0.5,188,32770,60,582,0.369352
6,0.6,156,32787,43,614,0.321981
7,0.7,132,32802,28,638,0.283871
8,0.8,113,32813,17,657,0.251111
9,0.9,88,32820,10,682,0.202765


In [53]:
for param in model.parameters():
    param.requires_grad = True

In [54]:
lrn.name = 'resnest50-mixed-spec-unfrozen-20210502-144336'

In [55]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-52


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-02 17:03:15 epoch:  30 train loss: 0.115589 train f1: 0.549723 valid loss: 0.123151 valid f1: 0.216257


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 